In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import print_function
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.tmp_framework import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *

VISUALIZE = False
graph = None
SAMPLED_DATA = defaultdict(dict)
UPDATE_DAT = True

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"

def main(dataset_list=None, N_retry_test = None):
    gtimer = GlobalTimer.instance()
    elog = Logger()
    crob = CombinedRobot(connection_list=(False, False))
    false_fail_accum = 0
    false_succ_accum = 0
    succ_accum = 0
    fail_accum = 0

    CHECK_DICT = {}

    # ## Load Global params
    if dataset_list is None:
        DATASET_LIST = sorted(filter(lambda x: x not in ["backup", "converted"], os.listdir(DATA_PATH)))
    else:
        DATASET_LIST = dataset_list
    for DATASET in DATASET_LIST:
        CHECK_DICT[DATASET] = {}
        CURRENT_PATH = os.path.join(DATA_PATH, DATASET)

        ## Load global params
        GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
        WDH = GLOBAL_PARAMS["WDH"]
        L_CELL = GLOBAL_PARAMS["L_CELL"]
        RATIO_DIMS = GLOBAL_PARAMS["RATIO_DIMS"]
        REACH_OFFSET_DICT = GLOBAL_PARAMS["REACH_OFFSET_DICT"]
        GRIPPER_REFS = GLOBAL_PARAMS["GRIPPER_REFS"]
        BASE_LINK = GLOBAL_PARAMS["BASE_LINK"]
        S_F_RATIO = GLOBAL_PARAMS["S_F_RATIO"]
        TIMEOUT = GLOBAL_PARAMS["TIMEOUT"]

        CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
        Ws, Ds, Hs = WDH
        Nw, Nd, Nh = Nwdh = int(Ws / L_CELL), int(Ds / L_CELL), int(Hs / L_CELL)
        L_MAX = L_CELL * RATIO_DIMS
        print("scene size: {} ({},{},{})".format(Nw * Nd * Nh, Nw, Nd, Nh))

        # ## Load world
        WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
        for WORLD in WORLD_LIST:
            CHECK_DICT[DATASET][WORLD] = {}
            WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
            SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
            Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
            reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

            cam = None
            # set urdf
            xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict,
                                                                               crob.custom_limits, start_rviz=VISUALIZE)
            ghnd = GeometryHandle(urdf_content)
            time.sleep(2)

            # set graph
            graph = TMPFramework(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                                    urdf_content=urdf_content, combined_robot=crob)
            graph.set_camera(cam)
            graph.set_cam_robot_collision(_add_cam_poles=False, color=(1, 1, 0, 0.3))
            if VISUALIZE: graph.set_rviz()

            # start UI
            ui_broker = UIBroker.instance()
            ui_broker.initialize(graph)
            ui_broker.start_server()

            # set rviz
            if VISUALIZE: graph.set_rviz(crob.home_pose)
            ui_broker.set_tables()

            for gripper in GRIPPER_REFS.values():
                graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0, 0, 0],
                                      link_name=gripper['link_name'])
            graph.register_binder(name='base', _type=PlaceFrame, point=[0, 0, 0], rpy=[0, 0, 0], link_name=BASE_LINK)
            vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                          dims=(0, 0, 0), center=(0, 0, 0), rpy=(0, 0, 0), collision=False, display=False
                                          )
            graph.add_object("virtual",
                             SingleHandleObject(_object=vtem,
                                                action_point=FramePoint(name="point", _object=vtem, point=(0, 0, 0),
                                                                        rpy=(0, 0, 0), name_full=None)),
                             binding=("point", "base"))

            obj_list = []
            col_obj_list = []

            # ## Load scene
            SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
            for SCENE in SCENE_LIST:
                CHECK_DICT[DATASET][WORLD][SCENE] = {}
                SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
                SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
                Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
                Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
                Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
                obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

                if VISUALIZE:
                    graph.show_pose(Q_s)
                    time.sleep(1)
                    graph.show_pose(Q_s)
                    time.sleep(1)
                    graph.show_pose(Q_s)
                for obj in obj_list: graph.remove_geometry(obj)
                for odat in obj_dat:
                    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], \
                                                            odat["color"], odat["center"], odat["rpy"]
                    obj = graph.ghnd.create_safe(
                        name="{}_{}_{}_{}".format(gtype.name, *nbox), link_name=BASE_LINK, gtype=gtype,
                        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
                    obj_list.append(obj)
                    graph.add_marker(obj, vis=VISUALIZE)

                for obj in col_obj_list: graph.remove_geometry(obj)

                if VISUALIZE: graph.set_rviz()
                dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
                if VISUALIZE: graph.set_rviz()

                # planners
                mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path,
                                      urdf_content=graph.urdf_content,
                                      robot_names=graph.combined_robot.robot_names,
                                      binder_links=[v.object.link_name for v in graph.binder_dict.values()],
                                      ghnd=graph.ghnd)
                dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path,
                                                        graph.link_names, graph.combined_robot.robot_names)

                # ## Load action
                ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
                for ACTION in ACTION_LIST:
                    print("[BEGIN] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION] = {}
                    snode_dict_bak = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
                    dcol.snode_dict = dcol.manager.dict()
                    for k, v in snode_dict_bak.items():
                        dcol.snode_dict[k] = deepcopy(v)
                    snode_keys = sorted(snode_dict_bak.keys())

                    if snode_keys:
                        sk0 = snode_keys[0]
                        rname, tar, inhand_coll, obj_coll = \
                            snode_dict_bak[sk0]["rname1"], snode_dict_bak[sk0]["rname2"], \
                            snode_dict_bak[sk0]["obj1"]["collision"], snode_dict_bak[sk0]["obj2"]["collision"]
                    else:
                        print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
                        print("No Item")
                        continue

                    dcol.check_loop_mp(test_full_mp, GRIPPER_REFS=GRIPPER_REFS,  Q_s=Q_s, dual_mplan_dict=dual_mplan_dict, mplan=mplan, N_retry=N_retry_test, timeout=TIMEOUT)
                    succ_old_list = np.array([snode_dict_bak[skey]["success"] for skey in snode_keys])
                    succ_now_list = np.array([dcol.snode_dict[skey]["succ_count"]>0 for skey in snode_keys])

                    if rname and tar:  # handover case
                        action_type = "HANDOVER"
                    elif inhand_coll:  # place case
                        action_type = "PLACE"
                    elif obj_coll:  # pick case
                        action_type = "PICK"
                    else:
                        raise (RuntimeError("non-implemented case"))

                    check_results = succ_old_list==succ_now_list
                    print(succ_old_list)
                    print(succ_now_list)
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["action_type"] = action_type
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["check_ratio"] = np.mean(check_results)
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["succ_corrects"] = np.mean(
                        check_results[np.where(succ_old_list)])
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["fail_corrects"] = np.mean(
                        check_results[np.where(np.logical_not(succ_old_list))])
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["num"] = len(check_results)
                    
                    false_fail_accum += np.sum(np.logical_not(check_results[np.where(np.logical_not(succ_old_list))]))
                    false_succ_accum += np.sum(np.logical_not(check_results[np.where(succ_old_list)]))
                    succ_accum += np.sum(succ_old_list)
                    fail_accum += np.sum(np.logical_not(succ_old_list))
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["false_fail_accum"] = false_fail_accum
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["false_succ_accum"] = false_succ_accum
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["succ_accum"] = succ_accum
                    CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["fail_accum"] = fail_accum

                    if UPDATE_DAT:
                        for isk, skey in zip(range(len(snode_keys)), snode_keys):
                            if not succ_old_list[isk] and succ_now_list[isk]:
                                snode_dict_bak[skey]["success"] = bool(succ_now_list[isk])
                        save_json(os.path.join(SCENE_PATH, ACTION), snode_dict_bak)

                    if VISUALIZE: graph.set_rviz()

                    print("[END] {} - {} - {} - {} ===============".format(DATASET, WORLD, SCENE, ACTION))
                    elog.log(CHECK_DICT[DATASET][WORLD][SCENE][ACTION], "{}-{}".format(SCENE, ACTION), print_now=False)
                    for k, v in CHECK_DICT[DATASET][WORLD][SCENE][ACTION].items():
                        print("{}: {}".format(k, str(v)))
    xcustom.clear()

connection_list
(False, False)
scene size: 3375 (15,15,15)


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
DATA CHECK SUCCESS: (True->True)
fail log
fail log
DATA CHECK SUCCESS: (True->True)
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK FAILURE: (True->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK FAILURE: (True->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (True->True)
fail log
fail

fail log
fail log
DATA CHECK SUCCESS: (False->False)
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fa

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK FAILURE: (True->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (True->True)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
fail log
DATA CHECK SUCCESS: (False->False)
fail log
fail

In [ ]:
xcustom.clear()

In [2]:
for k1, v1 in CHECK_DICT.items():
    print(k1)
    for k2, v2 in v1.items():
        print("\t"+k2)
        for k3, v3 in v2.items():
            print("\t\t"+k3)
            for k4, v4 in v3.items():
                print("\t\t\t"+k4)
                print("\t\t\t"+str(v4))

20201207-215956
	WORLD-20201207-220034
		SCENE-20201207-220036
			20201207-220101.json
			{'fail_corrects': 1.0, 'num': 23, 'succ_corrects': 1.0, 'action_type': 'PICK', 'check_ratio': 1.0}
			20201207-220049.json
			{'fail_corrects': 1.0, 'num': 13, 'succ_corrects': 0.2, 'action_type': 'HANDOVER', 'check_ratio': 0.6923076923076923}
			20201207-220112.json
			{'fail_corrects': 1.0, 'num': 25, 'succ_corrects': 0.7142857142857143, 'action_type': 'PLACE', 'check_ratio': 0.92}
	WORLD-20201207-215956
		SCENE-20201207-215958
			20201207-220022.json
			{'fail_corrects': 1.0, 'num': 2, 'succ_corrects': 1.0, 'action_type': 'PICK', 'check_ratio': 1.0}
			20201207-220034.json
			{'fail_corrects': 1.0, 'num': 24, 'succ_corrects': 1.0, 'action_type': 'PLACE', 'check_ratio': 1.0}
			20201207-220011.json
			{'fail_corrects': 1.0, 'num': 76, 'succ_corrects': 0.8888888888888888, 'action_type': 'HANDOVER', 'check_ratio': 0.9605263157894737}


In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from __future__ import print_function
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.tmp_framework import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *

VISUALIZE = False
graph = None
SAMPLED_DATA = defaultdict(dict)
UPDATE_DAT = True
N_retry_test = 10

gtimer = GlobalTimer.instance()
crob = CombinedRobot(connection_list=(False, False))

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"

CHECK_DICT = {}

# ## Load Global params
DATASET_LIST = sorted(os.listdir(DATA_PATH))
for DATASET in DATASET_LIST:
    CHECK_DICT[DATASET] = {}
    CURRENT_PATH = os.path.join(DATA_PATH, DATASET)
    GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
    for k, v in GLOBAL_PARAMS.items():
        locals()[k] = v

    CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
    Ws, Ds, Hs = WDH
    Nw, Nd, Nh = Nwdh = int(Ws / L_CELL), int(Ds / L_CELL), int(Hs / L_CELL)
    L_MAX = L_CELL * RATIO_DIMS
    print("scene size: {} ({},{},{})".format(Nw * Nd * Nh, Nw, Nd, Nh))

    # ## Load world
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    for WORLD in WORLD_LIST:
        CHECK_DICT[DATASET][WORLD] = {}
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
        Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
        reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

        cam = None
        # set urdf
        xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict,
                                                                           crob.custom_limits, start_rviz=VISUALIZE)
        ghnd = GeometryHandle(urdf_content)
        time.sleep(2)

        # set graph
        graph = TMPFramework(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                                urdf_content=urdf_content, combined_robot=crob)
        graph.set_camera(cam)
        graph.set_cam_robot_collision(_add_cam_poles=False, color=(1, 1, 0, 0.3))
        if VISUALIZE: graph.set_rviz()

        # start UI
        ui_broker = UIBroker.instance()
        ui_broker.initialize(graph)
        ui_broker.start_server()

        # set rviz
        if VISUALIZE: graph.set_rviz(crob.home_pose)
        ui_broker.set_tables()

        for gripper in GRIPPER_REFS.values():
            graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0, 0, 0],
                                  link_name=gripper['link_name'])
        graph.register_binder(name='base', _type=PlaceFrame, point=[0, 0, 0], rpy=[0, 0, 0], link_name=BASE_LINK)
        vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                      dims=(0, 0, 0), center=(0, 0, 0), rpy=(0, 0, 0), collision=False, display=False
                                      )
        graph.add_object("virtual",
                         SingleHandleObject(_object=vtem,
                                            action_point=FramePoint(name="point", _object=vtem, point=(0, 0, 0),
                                                                    rpy=(0, 0, 0), name_full=None)),
                         binding=("point", "base"))

        obj_list = []
        col_obj_list = []

        # ## Load scene
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            CHECK_DICT[DATASET][WORLD][SCENE] = {}
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
            Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
            Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
            Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
            obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

            if VISUALIZE:
                graph.show_pose(Q_s)
                time.sleep(1)
                graph.show_pose(Q_s)
                time.sleep(1)
                graph.show_pose(Q_s)
            for obj in obj_list: graph.remove_geometry(obj)
            for odat in obj_dat:
                nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], \
                                                        odat["color"], odat["center"], odat["rpy"]
                obj = graph.ghnd.create_safe(
                    name="{}_{}_{}_{}".format(gtype.name, *nbox), link_name=BASE_LINK, gtype=gtype,
                    center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
                obj_list.append(obj)
                graph.add_marker(obj, vis=VISUALIZE)

            if "col_obj_list" in locals():
                for obj in col_obj_list: graph.remove_geometry(obj)

            if VISUALIZE: graph.set_rviz()
            dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
            if VISUALIZE: graph.set_rviz()

            # planners
            mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path,
                                  urdf_content=graph.urdf_content,
                                  robot_names=graph.combined_robot.robot_names,
                                  binder_links=[v.object.link_name for v in graph.binder_dict.values()],
                                  ghnd=graph.ghnd)
            dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path,
                                                    graph.link_names, graph.combined_robot.robot_names)

            # ## Load action
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION] = {}
                dcol.snode_dict = {int(k): v for k, v in load_json(os.path.join(SCENE_PATH, ACTION)).items()}
                #                 print([dcol.snode_dict[skey]['success'] for skey in sorted(dcol.snode_dict.keys())])

                check_results = []
                succ_records = []
                succ_nows = []
                action_type = ""
                for skey in sorted(dcol.snode_dict.keys()):
                    snode = dcol.snode_dict[skey]
                    rname, inhand, obj, tar, dims_bak, color_bak, succ, _ = load_manipulation_from_dict(snode,
                                                                                                        graph.ghnd)
                    success_now = False
                    for _ in range(N_retry_test):
                        if rname and tar:  # handover case
                            trajectory, Q_last, error, success_now = test_handover(graph, GRIPPER_REFS, rname, inhand,
                                                                                   obj, tar, Q_s,
                                                                                   dual_mplan_dict[(rname, tar)])
                            remove_map = [[], [0, 1]]
                            action_type = "HANDOVER"
                        elif inhand.collision:  # place case
                            trajectory, Q_last, error, success_now = test_place(graph, GRIPPER_REFS, rname, inhand, obj,
                                                                                tar, Q_s, mplan)
                            remove_map = [[], [0, 1]]
                            action_type = "PLACE"
                        elif obj.collision:  # pick case
                            trajectory, Q_last, error, success_now = test_pick(graph, GRIPPER_REFS, rname, inhand, obj,
                                                                               tar, Q_s, mplan)
                            remove_map = [[1], [0]]
                            action_type = "PICK"
                        else:
                            raise (RuntimeError("non-implemented case"))
                        if success_now:
                            break
                        else:
                            print("fail log")
                    check_result = succ == success_now
                    print("DATA CHECK {}: ({}->{})".format("SUCCESS" if check_result else "FAILURE", succ, success_now))
                    check_results.append(check_result)
                    succ_records.append(succ)
                    succ_nows.append(success_now)
                    if UPDATE_DAT and success_now and not succ:
                        dcol.snode_dict[skey]['success'] = success_now

                    if VISUALIZE and success_now:
                        show_manip_coords(graph, GRIPPER_REFS, action_type, rname, inhand, obj, rname2=tar)
                        graph.show_motion(trajectory, period=0.1)

                    remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
                    remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
                    reset_rendering(graph, action_type, remove1, remove2, dims_bak, color_bak, sleep=True,
                                    vis=VISUALIZE)
                check_results = np.array(check_results)
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["action_type"] = action_type
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["check_ratio"] = np.mean(check_results)
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["succ_corrects"] = np.mean(
                    check_results[np.where(succ_records)])
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["fail_corrects"] = np.mean(
                    check_results[np.where(np.logical_not(succ_records))])
                CHECK_DICT[DATASET][WORLD][SCENE][ACTION]["num"] = len(check_results)

                if UPDATE_DAT: save_json(os.path.join(SCENE_PATH, ACTION), dcol.snode_dict)

                if VISUALIZE: graph.set_rviz()


In [ ]:
for k1, v1 in CHECK_DICT.items():
    print(k1)
    for k2, v2 in v1.items():
        print("\t"+k2)
        for k3, v3 in v2.items():
            print("\t\t"+k3)
            for k4, v4 in v3.items():
                print("\t\t\t"+k4)
                print("\t\t\t"+str(v4))

# Part-by-part

In [ ]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt
from pkg.marker_config import *
from pkg.tmp_framework import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *
from pkg.controller.combined_robot import CombinedRobot, XYZ_RPY_ROBOTS_DEFAULT
from pkg.data_collecting.sampling import *


VISUALIZE = True
graph = None
SAMPLED_DATA = defaultdict(dict)
N_retry_test = 10

gtimer = GlobalTimer.instance()
crob = CombinedRobot(connection_list=(False, False))

GLOBAL_FILENAME = "global.json"
WORLD_FILENAME = "world.json"
SCENE_FILENAME = "scene.json"
DATA_PATH = "./data"
try: os.mkdir(DATA_PATH)
except: pass

## Load Global params

In [ ]:
DATASET_LIST = sorted(os.listdir(DATA_PATH))

In [ ]:
CURRENT_PATH = os.path.join(DATA_PATH, DATASET_LIST[0])
GLOBAL_PARAMS = load_json(os.path.join(CURRENT_PATH, GLOBAL_FILENAME))
for k, v in GLOBAL_PARAMS.items():
    locals()[k] = v

In [ ]:
CENTER = tuple(np.divide(WDH, 2, dtype=np.float))
Ws, Ds, Hs = WDH 
Nw, Nd, Nh = Nwdh = int(Ws/L_CELL), int(Ds/L_CELL), int(Hs/L_CELL)
L_MAX = L_CELL*RATIO_DIMS
print("scene size: {} ({},{},{})".format(Nw*Nd*Nh, Nw, Nd, Nh))

## Load world

In [ ]:
WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))

In [ ]:
WORLD_PATH = os.path.join(CURRENT_PATH, WORLD_LIST[0])
SAMPLED_DATA["WORLD"] = load_json(os.path.join(WORLD_PATH, WORLD_FILENAME))
Trbt_dict = SAMPLED_DATA["WORLD"]["Trbt_dict"]
reach_center_dict = {k: tuple(np.add(v[0], REACH_OFFSET_DICT[k])) for k, v in Trbt_dict.items()}

In [ ]:
cam = None
# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, Trbt_dict, crob.custom_limits, start_rviz=VISUALIZE)
ghnd = GeometryHandle(urdf_content)
time.sleep(2)
# set graph
if graph is None:
    graph = TMPFramework(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                            urdf_content=urdf_content, combined_robot=crob)
    graph.set_camera(cam)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()

    # start UI
    ui_broker = UIBroker(graph)
    ui_broker.start_server()

    # set rviz
    if VISUALIZE: graph.set_rviz(crob.home_pose)
    ui_broker.set_tables()
else:
    graph.clear_markers()
    graph.clear_highlight()
    graph.ghnd.clear()
    graph.__init__(ghnd=graph.ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES,
                   urdf_content=urdf_content, combined_robot=crob)
    graph.set_cam_robot_collision(_add_cam_poles=False, color=(1,1,0,0.3))
    if VISUALIZE: graph.set_rviz()
        
for gripper in GRIPPER_REFS.values():
    graph.register_binder(name=gripper['bname'], _type=FramedTool, point=gripper['tcp_ref'], rpy=[0,0,0], link_name=gripper['link_name'])
graph.register_binder(name='base', _type=PlaceFrame, point=[0,0,0], rpy=[0,0,0], link_name=BASE_LINK)
vtem = graph.ghnd.create_safe(name="virtual", gtype=GEOTYPE.SPHERE, link_name=BASE_LINK,
                                             dims=(0,0,0), center=(0,0,0), rpy=(0,0,0), collision=False, display=False
                                            )
graph.add_object("virtual",
                 SingleHandleObject(_object=vtem,
                                   action_point=FramePoint(name="point", _object=vtem, point=(0,0,0), rpy=(0,0,0), name_full=None)),
                 binding=("point", "base"))

obj_list=[]
col_obj_list=[]

## Load scene

In [ ]:
SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))

In [ ]:
SCENE_PATH = os.path.join(WORLD_PATH, SCENE_LIST[0])
SAMPLED_DATA["SCENE"] = load_json(os.path.join(SCENE_PATH, SCENE_FILENAME))
Q_s = np.array(SAMPLED_DATA["SCENE"]["Q_s"])
Q_s, links, link_verts, link_ctems, link_rads = sample_joint(graph, Q_s_loaded=Q_s)
Q_s_dict = SAMPLED_DATA["SCENE"]["Q_s_dict"]
obj_dat = SAMPLED_DATA["SCENE"]["obj_dat"]

In [ ]:
if VISUALIZE:
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
    time.sleep(1)
    graph.show_pose(Q_s)
for obj in obj_list: graph.remove_geometry(obj)
for odat in obj_dat:
    nbox, gtype, dims, color, center, rpy = odat["nbox"], getattr(GEOTYPE, odat["gtype"]), odat["dims"], odat["color"], odat["center"], odat["rpy"]
    obj = graph.ghnd.create_safe(
        name="{}_{}_{}_{}".format(gtype.name,*nbox), link_name=BASE_LINK, gtype=gtype,
        center=center, rpy=rpy, dims=dims, color=color, display=True, collision=True, fixed=True)
    obj_list.append(obj)
    graph.add_marker(obj, vis=VISUALIZE)

if "col_obj_list" in locals():
    for obj in col_obj_list: graph.remove_geometry(obj)

if VISUALIZE: graph.set_rviz()
dcol = DataCollector(graph, GRIPPER_REFS, S_F_RATIO=S_F_RATIO)
if VISUALIZE: graph.set_rviz()

# planners
mplan = MoveitPlanner(joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content,
                      robot_names=graph.combined_robot.robot_names, binder_links=[v.object.link_name for v in graph.binder_dict.values()], ghnd=graph.ghnd)
dual_mplan_dict = get_dual_planner_dict(GRIPPER_REFS, graph.ghnd, graph.urdf_content, graph.urdf_path, graph.link_names, graph.combined_robot.robot_names)

## Load action

In [ ]:
ACTION_LIST = sorted(filter(lambda x: x!=SCENE_FILENAME, os.listdir(SCENE_PATH)))

In [ ]:
dcol.snode_dict = {int(k):v for k,v in load_json(os.path.join(SCENE_PATH, ACTION_LIST[1])).items()}

In [ ]:
check_results = []
succ_records = []
succ_nows = []
for skey in sorted(dcol.snode_dict.keys()):
    snode = dcol.snode_dict[skey]
    rname, inhand, obj, tar, dims_bak, color_bak, succ, _ = load_manipulation_from_dict(snode, graph.ghnd)
    success_now = False
    for _ in range(N_retry_test):
        if rname and tar: # handover case
            trajectory, Q_last, error, success_now = test_handover(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, dual_mplan_dict[(rname, tar)])
            remove_map = [[], [0,1]]
        elif inhand.collision: # place case
            trajectory, Q_last, error, success_now = test_place(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, mplan)
            remove_map = [[], [0,1]]
        elif obj.collision: # pick case
            trajectory, Q_last, error, success_now = test_pick(graph, GRIPPER_REFS, rname, inhand, obj, tar, Q_s, mplan)
            remove_map = [[1], [0]]
        else:
            raise(RuntimeError("non-implemented case"))
        if success_now:
            break
        else:
            print("fail log")
    check_result = succ == success_now
    print("DATA CHECK {}: ({}->{})".format("SUCCESS" if check_result else "FAILURE", succ, success_now))
    check_results.append(check_result)
    succ_records.append(succ)
    succ_nows.append(success_now)

    if VISUALIZE and success_now:
        show_manip_coords(graph, GRIPPER_REFS, "TESTING", rname, inhand, obj, rname2=tar)
        graph.show_motion(trajectory, period=0.1)

    remove1 = [[inhand, obj][iii] for iii in remove_map[0]]
    remove2 = [[inhand, obj][iii] for iii in remove_map[1]]
    reset_rendering(graph, "TESTING", remove1, remove2, dims_bak, color_bak, sleep=True, vis=True)

In [ ]:
graph.set_rviz()